In [1]:
# Importing Packages
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import torch
import torch.nn as nn
import mediapipe as mp
from collections import deque, Counter
import math

# Defining LSTM Model Class
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Preprocess Angles Function
def preprocess_angles(angles):
    angles_tensor = torch.tensor(angles, dtype=torch.float32)
    angles_tensor = angles_tensor.unsqueeze(0).unsqueeze(0)
    return angles_tensor


# Calculate 3D Angle Function
def calculate_angle_3d(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c) 

    vector1 = a - b
    vector2 = c - b

    dot_product = np.dot(vector1, vector2)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)

    cosine_theta = dot_product / (magnitude1 * magnitude2)
    
    # Calculate the angle in radians
    radians = np.arccos(np.clip(cosine_theta, -1.0, 1.0))

    angle = np.degrees(radians)

    return angle

# Class Labels and Names
class_labels = [0, 1, 2, 3, 4]
class_names = ['Squat', 'Push up', 'Crunch', 'Lunge', 'Leg raise']

In [2]:
# Debouncing
N = 10  # Number of frames to consider for debouncing
predicted_classes_buffer = deque(maxlen=N)
angle_buffer = deque(maxlen=10)

# crunch count variable
line_a = None
line_b = None
angle_degrees=0

input_size = 8
hidden_size = 64
num_layers = 2
output_size = 5

# Model Loading and Initialization
loaded_model = LSTMModel(input_size, hidden_size, num_layers, output_size)
loaded_model.load_state_dict(torch.load('model/lstm_model.pth'))
loaded_model.eval()

# MediaPipe Pose Initialization
mp_pose = mp.solutions.pose
cap = cv2.VideoCapture(0)

# Count variable
squat_count = 0 
lunge_count = 0
legraise_count = 0
pushup_count = 0
crunch_count = 0

legraise_stage = None
lunge_stage = None
squat_stage = None
pushup_stage = None
crunch_stage = None


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            keypoints_xyz = [(landmarks[i].x, landmarks[i].y, landmarks[i].z) for i in [12, 14, 16, 12, 24, 14, 24, 26, 12, 24, 26, 28, 11, 13, 15, 11, 23, 13, 23, 25, 11, 23, 25, 27]]
            
            angles = []  # List to store angles for this frame

            for i in range(0, len(keypoints_xyz), 3):
                x1, y1, z1 = keypoints_xyz[i]
                x2, y2, z2 = keypoints_xyz[i + 1]
                x3, y3, z3 = keypoints_xyz[i + 2]

                angle_rad = math.atan2(z2 - z1, y2 - y1) - math.atan2(z3 - z2, y3 - y2)
                angle_deg = math.degrees(angle_rad)
                angles.append(angle_deg)


                cv2.line(frame, (int(x1 * frame.shape[1]), int(y1 * frame.shape[0])),
                         (int(x2 * frame.shape[1]), int(y2 * frame.shape[0])), (0, 255, 0), 2)
                cv2.line(frame, (int(x2 * frame.shape[1]), int(y2 * frame.shape[0])),
                         (int(x3 * frame.shape[1]), int(y3 * frame.shape[0])), (0, 255, 0), 2)
                cv2.putText(frame, f'Angle {i // 3 + 1}: {angle_deg:.2f}', (int(x1 * frame.shape[1]), int(y1 * frame.shape[0]) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            angle_buffer.append(angles)


        cv2.rectangle(frame, (0,0), (900,150), (256,256,256), -1) 

        if len(angle_buffer) == 10:
            sequence_tensor = preprocess_angles(angle_buffer)
            sequence_tensor = sequence_tensor.view(1, 10, 8)  # Reshape to (1, 10, 8)

            # Predict motion using the LSTM model
            with torch.no_grad():
                prediction = loaded_model(sequence_tensor)

            predicted_class = torch.argmax(prediction, dim=1).item()
            predicted_classes_buffer.append(predicted_class)
            final_predicted_class = Counter(predicted_classes_buffer).most_common(1)[0][0]
            predicted_name = class_names[final_predicted_class]

            # cv2.putText(frame, f'Predicted Motion: {predicted_name} ({predicted_percentage:.2%})', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f'Predicted Motion:', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (55,14,15), 2,cv2.LINE_AA)
            cv2.putText(frame, f'{predicted_name}', (600, 60), cv2.FONT_HERSHEY_DUPLEX, 2, (6,84,226), 2,cv2.LINE_AA)


            for i, prob in enumerate(torch.softmax(prediction, dim=1)[0]):
                class_name = class_names[i]
                probability = prob.item()
                text = f'{class_name}: {probability:.2%}'
                # cv2.putText(frame, text, (10, 60 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)


            # Push up Count
            if predicted_name == 'Push up':
                    left_shoulder = keypoints_xyz[12]
                    right_shoulder = keypoints_xyz[0]
                    right_elbow = keypoints_xyz[1]
                    right_wrist = keypoints_xyz[2]

                    pushup_angle = calculate_angle_3d(right_shoulder, right_elbow, right_wrist)
                    # print(pushup_angle)
                    
                    cv2.putText(frame, f"Right Angle: {pushup_angle:.2f}",
                                (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

                    if pushup_angle > 150:
                        pushup_stage = "up"
                    if pushup_angle < 95 and pushup_stage == 'up':
                        pushup_stage = "down"
                        pushup_count += 1
                    cv2.putText(frame, f"Count:", (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,14,15), 2, cv2.LINE_AA)
                    cv2.putText(frame, f"{pushup_count}", (200, 120), cv2.FONT_HERSHEY_DUPLEX, 1, (6,84,226), 2, cv2.LINE_AA)


            # Leg raise Count
            if predicted_name == 'Leg raise':
                    left_shoulder = keypoints_xyz[12]
                    left_hip = keypoints_xyz[21]
                    left_knee = keypoints_xyz[22]

                    legraise_angle = calculate_angle_3d(left_shoulder, left_hip, left_knee)
                    # print(legraise_angle)
                    
                    cv2.putText(frame, f"Left Angle: {legraise_angle:.2f}",
                                (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

                    if legraise_angle < 110:
                        legraise_stage = "up"
                    if legraise_angle > 140 and legraise_stage == "up":
                        legraise_stage = "down"
                        legraise_count += 1
                    cv2.putText(frame, f"Count:", (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,14,15), 2, cv2.LINE_AA)
                    cv2.putText(frame, f"{legraise_count}", (200, 120), cv2.FONT_HERSHEY_DUPLEX, 1, (6,84,226), 2, cv2.LINE_AA)

            # Crunch Count
            if predicted_name == 'Crunch':
                
                if results.pose_landmarks:
                    c_landmarks = results.pose_landmarks.landmark

                    point5 = (int(c_landmarks[5].x * frame.shape[1]), int(landmarks[5].y * frame.shape[0]))
                    point10 = (int(c_landmarks[10].x * frame.shape[1]), int(landmarks[10].y * frame.shape[0]))
                    point12 = (int(c_landmarks[12].x * frame.shape[1]), int(landmarks[12].y * frame.shape[0]))
                    point24 = (int(c_landmarks[24].x * frame.shape[1]), int(landmarks[24].y * frame.shape[0]))
                    point30 = (int(c_landmarks[30].x * frame.shape[1]), int(landmarks[30].y * frame.shape[0]))
                    point32 = (int(c_landmarks[32].x * frame.shape[1]), int(landmarks[32].y * frame.shape[0]))

                    if line_a is not None:
                        cv2.line(frame, line_a[0], line_a[1], (0, 0, 255), 2, cv2.LINE_AA)

                    if line_b is not None:
                        cv2.line(frame, line_b[0], line_b[1], (0, 0, 255), 2, cv2.LINE_AA)

                    if line_a is not None and line_b is not None:
                        vector_a = np.array(line_a[1]) - np.array(line_a[0])
                        vector_b = np.array(line_b[1]) - np.array(line_b[0])
                        dot_product = np.dot(vector_a, vector_b)
                        norm_a = np.linalg.norm(vector_a)
                        norm_b = np.linalg.norm(vector_b)
                        angle = np.arccos(dot_product / (norm_a * norm_b))
                        angle_degrees = math.degrees(angle)
                        cv2.putText(frame, f'Angle: {angle_degrees:.2f} degrees', (990, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)    

                    line_a = (point5, point12)
                    line_b = (point12, point24)

                    if angle_degrees < 1 :
                        crunch_stage = "down"
                    if angle_degrees < 25 and crunch_stage == "down":
                        crunch_stage = "up"
                        crunch_count += 1
   
                    cv2.putText(frame, f"Count:", (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,14,15), 2, cv2.LINE_AA)
                    cv2.putText(frame, f"{crunch_count} {crunch_stage}", (200, 120), cv2.FONT_HERSHEY_DUPLEX, 1, (6,84,226), 2, cv2.LINE_AA)
                    

            # Lunge Count
            if predicted_name == 'Lunge':
                    left_shoulder = keypoints_xyz[12]
                    left_hip = keypoints_xyz[21]
                    left_knee = keypoints_xyz[22]
                    left_ankle = keypoints_xyz[23]

                    lunge_angle = calculate_angle_3d(left_hip, left_knee, left_ankle)
                    # print(lunge_angle)
                    
                    cv2.putText(frame, f"Left Angle: {lunge_angle:.2f}",
                                (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

                    if lunge_angle < 110:
                        lunge_stage = "down"
                    if lunge_angle > 140 and lunge_stage == 'down':
                        lunge_count += 1
                        lunge_stage = "up"
                        
                    cv2.putText(frame, f"Count:", (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,14,15), 2, cv2.LINE_AA)
                    cv2.putText(frame, f"{lunge_count}", (200, 120), cv2.FONT_HERSHEY_DUPLEX, 1, (6,84,226), 2, cv2.LINE_AA)


            # Squat Count
            if predicted_name == 'Squat':
                    left_shoulder = keypoints_xyz[12]
                    left_hip = keypoints_xyz[21]
                    left_knee = keypoints_xyz[22]
                    left_ankle = keypoints_xyz[23]
                    
                    squat_angle = calculate_angle_3d(left_hip, left_knee, left_ankle)
                    # print(squat_angle)
                    
                    cv2.putText(frame, f"Left Angle: {squat_angle:.2f}",
                                (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

                    if squat_angle < 45:
                        squat_stage = "down"
                    if squat_angle > 120 and squat_stage == 'down':
                        squat_count += 1
                        squat_stage = "up"

                    cv2.putText(frame, f"Count:", (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,14,15), 2, cv2.LINE_AA)
                    cv2.putText(frame, f"{squat_count}", (200, 120), cv2.FONT_HERSHEY_DUPLEX, 1, (6,84,226), 2, cv2.LINE_AA)


        else:
            predicted_class ="detecting" 

            cv2.putText(frame, f'Predicted Motion:', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (55,14,15), 2,cv2.LINE_AA)
            cv2.putText(frame, f'{predicted_class}', (600, 60), cv2.FONT_HERSHEY_DUPLEX, 2, (6,84,226), 2,cv2.LINE_AA)
        
       
    cv2.imshow('Real-time Motion Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
        break


cap.release()
cv2.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
